In [1]:
import pandas as pd
import numpy as np

In [2]:
file = pd.read_csv('events_up_to_01062018.csv',low_memory=False)

In [9]:
file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2341681 entries, 0 to 2341680
Data columns (total 23 columns):
timestamp                   object
event                       object
person                      object
url                         object
sku                         float64
model                       object
condition                   object
storage                     object
color                       object
skus                        object
search_term                 object
staticpage                  object
campaign_source             object
search_engine               object
channel                     object
new_vs_returning            object
city                        object
region                      object
country                     object
device_type                 object
screen_resolution           object
operating_system_version    object
browser_version             object
dtypes: float64(1), object(22)
memory usage: 410.9+ MB


In [3]:
gbp = file.groupby('person')

In [4]:
gbp

In [63]:
features_04 = pd.read_csv('features_04.csv')

## Primer evento

In [60]:
first_event = gbp.agg({'event':'first'})
first_event.columns = ['primer_evento']

## Modelos de celular distintos que vio

In [12]:
def modelosdistintos (x):
    return x.loc[x.event=='viewed product']['model'].nunique()
mod = gbp.apply(modelosdistintos)

## Cuantas veces hizo cada evento

In [16]:
events = ['searched products', 'ad campaign hit','checkout', 'search engine hit','generic listing', 'brand listing']
def cadaevento (x):
    l = []
    for e in events:
        l.append (x.loc[x.event==e,'event'].size)
    return pd.Series(l)
cadaevento = gbp.apply(cadaevento)

In [18]:
cadaevento.columns = ['busquedas','clicks_en_ads','checkouts','entradas_desde_motor','listar_productos','listar_marca']

## Tamaño de pantalla numerico (ancho x alto)

In [71]:
tampant = features_04.scrnres.str.split('x').apply(lambda x: (int(x[0])*int(x[1])) if x[0]!='Ninguno' else 0  )

## Entro desde 'direct'

In [72]:
direct = features_04['primer_channel'] == 'Direct'

## Entro desde 'Computer'

In [74]:
computer = features_04['dispos'] == 'Computer'

In [87]:
features_05 = features_04.set_index('person').join(first_event,how='left')

In [88]:
features_05['modelos_distintos'] = mod
features_05 = features_05.join(cadaevento,how='left')
features_05['tamanio_pantalla'] = tampant
features_05['entro_direct'] = direct
features_05['uso_computer'] = computer

In [89]:
features_05.head()

,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,primer_visita,ultima_visita,cant_eventos,permanencia,...,modelos_distintos,busquedas,clicks_en_ads,checkouts,entradas_desde_motor,listar_productos,listar_marca,tamanio_pantalla,entro_direct,uso_computer
person,,,,,,,,,,,,,,,,,,,,,
0008ed71,checkout,Ninguno,Computer,0,2,0,2018-05-17 12:27:47,2018-05-17 16:28:37,6,1,...,0,0,0,3,0,1,0,2073600,False,True
00091926,viewed product,iPhone 6S,Computer,0,34,372,2018-05-03 22:08:29,2018-05-31 19:52:03,448,28,...,36,0,15,2,0,0,25,786432,True,True
00091a7a,brand listing,iPhone 6S,Smartphone,0,1,3,2018-03-26 14:51:11,2018-03-26 14:56:58,10,1,...,3,0,1,0,0,0,5,230400,False,False
000ba417,viewed product,Samsung Galaxy A3 Duos,Computer,1,6,153,2018-05-17 11:11:45,2018-05-26 13:09:22,206,10,...,26,0,1,6,1,14,24,786432,True,True
000c79fe,searched products,iPhone 7,Smartphone,0,1,3,2018-05-29 00:27:47,2018-05-29 00:38:07,17,1,...,1,9,1,1,1,1,0,230400,False,False


In [90]:
features_05.to_csv('features_05.csv')